# Problem Statement

The vanilla Viterbi algorithm generally faces a high loss of accuracy majorly due to the fact that when the algorithm encounters an unknown word which was not present in the training set, it assigns an incorrect tag arbitrarily. This is because, for unknown words, the emission probabilities for all candidate tags are 0, so the algorithm arbitrarily chooses (the first) tag.

Using the Treebank dataset of NLTK with the 'universal' tagset, the Viterbi algorithm should be modified to solve the problem of unknown words using at least two techniques. The main objectives are:

- To write the vanilla Viterbi algorithm for assigning POS tags (i.e. without dealing with unknown words)
- To solve the problem of unknown words using at least two techniques. These techniques can use any of the approaches discussed in the class - lexicon, rule-based, probabilistic etc.
- To compare the tagging accuracy after making these modifications with the vanilla Viterbi algorithm.
- To list down at least three cases from the sample test file (i.e. unknown word-tag pairs) which were incorrectly tagged by the original Viterbi POS tagger and got corrected after your modifications.

## POS tagging using modified Viterbi

The steps that have been followed are mentioned below:

1. Build the vanilla Viterbi based POS tagger
2. Build Modified Viterbi I using probabilistic technique
3. Build Modified Viterbi II using rule-based backoff method

### Data Preparation

In [1]:
#Importing libraries
import nltk
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import pprint, time

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(len(nltk_data))
print(nltk_data[:3])

3914
[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'

In [3]:
#splitting the data into train and test sets
train_data, test_data = train_test_split(nltk_data, train_size = 0.95, test_size = 0.05, random_state = 1234)
print(len(train_data))
print(len(test_data))

3718
196


In [4]:
print(train_data[:3])

[[('This', 'DET'), ('year', 'NOUN'), (',', '.'), ('the', 'DET'), ('average', 'NOUN'), ('of', 'ADP'), ('daily', 'ADJ'), ('contracts', 'NOUN'), ('traded', 'VERB'), ('*', 'X'), ('totaled', 'VERB'), ('9,118', 'NUM'), (',', '.'), ('up', 'ADP'), ('from', 'ADP'), ('4,645', 'NUM'), ('a', 'DET'), ('year', 'NOUN'), ('earlier', 'ADJ'), ('and', 'CONJ'), ('from', 'ADP'), ('917', 'NUM'), ('in', 'ADP'), ('1984', 'NUM'), ('.', '.')], [('First', 'NOUN'), ('of', 'ADP'), ('America', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-1', 'X'), ('now', 'ADV'), ('has', 'VERB'), ('45', 'NUM'), ('banks', 'NOUN'), ('and', 'CONJ'), ('$', '.'), ('12.5', 'NUM'), ('billion', 'NUM'), ('*U*', 'X'), ('in', 'ADP'), ('assets', 'NOUN'), (',', '.'), ('announced', 'VERB'), ('an', 'DET'), ('agreement', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('acquire', 'VERB'), ('the', 'DET'), ('Peoria', 'NOUN'), (',', '.'), ('Ill.', 'NOUN'), (',', '.'), ('bank', 'NOUN'), ('holding', 'VERB'), ('company', 'NOUN'), ('in', 'ADP'), ('January', 'NOUN'),

In [5]:
# number of tagged words in train set
train_tagged_words = [tup for sent in train_data for tup in sent]
len(train_tagged_words)

95799

In [6]:
# checking the words present and the length of the vocabulary
tokens = [x[0] for x in train_tagged_words]
print(tokens[:10])

V = set(tokens)
print(len(V))

['This', 'year', ',', 'the', 'average', 'of', 'daily', 'contracts', 'traded', '*']
12073


In [7]:
# number of tags
T = set([x[1] for x in train_tagged_words])
print(len(T))
print(T)

12
{'X', 'CONJ', 'NUM', 'PRT', 'DET', 'ADP', '.', 'PRON', 'NOUN', 'ADJ', 'ADV', 'VERB'}


### Build the vanilla Viterbi based POS tagger

Based on the Hidden Markov Model (HMM) algorithm, we can assign a tag (t) to a word (w) such that the likelihood of P(t/w) is maximised.
We know that P(t/w) = P(w/t) * P(t) / P(w)
Here,
- P(w/t): Emission probability of a word (w) when the tag (t) is given. This can be calculated using the given formula:
 - P(w/t) = count(w, t) / count(t).
- P(t): Transition probability or the probability of the tag (t). As per the Markov order 1 assumption, the tag (t) of a word depends of the tag of the previous word or t(n-1).

##### Function to determine emission probability of a word when tag is given

In [8]:
def w_given_t(word, tag, train_bag = train_tagged_words):
    tag_pairs = [(w,t) for w,t in train_bag if t == tag]
    count_tag = len(tag_pairs)
    count_w_given_t = len([w for w,t in tag_pairs if w == word])
    
    return (count_w_given_t, count_tag)

##### Function to calculate transition probability of a tag when previous tag is given

In [9]:
def t1_given_t2(t1,t2,train_bag = train_tagged_words):
    tag_list = [t for w,t in train_bag]
    t1_tag_list = [x for x in tag_list if x == t2]
    count_t1 = len(t1_tag_list)
    count_t1_given_t2 = len([tag_list[i+1] for i in range(len(tag_list)-1) if tag_list[i] == t2 and tag_list[i+1] == t1])
    
    return (count_t1_given_t2, count_t1)

In [10]:
# checking the output of the w_given_t function from a tagged pair in the train set 
w_given_t('company', 'NOUN')

(252, 27471)

In [11]:
# checking the output of the t1_given_t2 function from the tags present in the train set
t1_given_t2('NOUN', 'ADJ')

(4235, 6063)

In [12]:
# creating T x T matrix named M[x, y] for to store P(t(y) given t(x))
M = np.zeros((len(T), len(T)), dtype = 'float32')

for x, t2 in enumerate(list(T)):
    for y, t1 in enumerate(list(T)): 
        M[x, y] = t1_given_t2(t1, t2)[0]/t1_given_t2(t1, t2)[1]

# converting matrix into dataframe for better interpretability
df_tag = pd.DataFrame(M, columns = list(T), index=list(T))
df_tag

,X,CONJ,NUM,PRT,DET,ADP,.,PRON,NOUN,ADJ,ADV,VERB
X,0.074842,0.010759,0.002690,0.184652,0.054114,0.144937,0.162816,0.056329,0.062184,0.016456,0.026108,0.204114
CONJ,0.008862,0.000466,0.041511,0.005131,0.118470,0.052705,0.033116,0.057369,0.348881,0.118937,0.055970,0.158582
NUM,0.210464,0.013377,0.184899,0.027051,0.002973,0.035672,0.115933,0.001486,0.354637,0.032402,0.002973,0.018133
PRT,0.014007,0.002280,0.056678,0.001954,0.099674,0.021173,0.041694,0.017915,0.247883,0.084039,0.009772,0.402932
DET,0.046197,0.000484,0.022373,0.000242,0.005442,0.009191,0.017777,0.003749,0.638650,0.203652,0.012698,0.039545
ADP,0.035048,0.000959,0.062001,0.001491,0.322893,0.016512,0.039842,0.070203,0.322893,0.105785,0.013849,0.008522
.,0.027314,0.057772,0.080593,0.002336,0.173226,0.090386,0.094070,0.065768,0.223091,0.044654,0.051932,0.088769
PRON,0.093929,0.004582,0.007255,0.011837,0.009164,0.023291,0.040473,0.007637,0.207331,0.073692,0.032837,0.487972
NOUN,0.029231,0.042263,0.009537,0.043974,0.013250,0.177023,0.239307,0.004769,0.264898,0.012231,0.017182,0.146336
ADJ,0.021442,0.016658,0.021112,0.010886,0.004948,0.077519,0.065809,0.000660,0.698499,0.065314,0.004948,0.012205


### Vanilla Viterbi based POS tagger

In [13]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([tag for word,tag in train_bag]))
    
    for key, word in enumerate(words):
        p = [] # initialising list of probability column for a given observation
        for tag in T:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]
                
            # calculating emission probability and state probability
            emission_p = w_given_t(words[key], tag)[0]/w_given_t(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

##### Testing Vanilla Viterbi POS Tagger on random sentences from test set

In [48]:
random.seed(1234)

# choosing 5 random sentences
rndom = [random.randint(1,len(test_data)) for x in range(5)]

# list of sentences
test_run = [test_data[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('``', '.'),
  ('When', 'ADV'),
  ('the', 'DET'),
  ('sell', 'NOUN'),
  ('programs', 'NOUN'),
  ('hit', 'VERB'),
  ('*T*-1', 'X'),
  (',', '.'),
  ('you', 'PRON'),
  ('can', 'VERB'),
  ('hear', 'VERB'),
  ('the', 'DET'),
  ('order', 'NOUN'),
  ('printers', 'NOUN'),
  ('start', 'VERB'),
  ('*-2', 'X'),
  ('to', 'PRT'),
  ('go', 'VERB'),
  ("''", '.'),
  ('on', 'ADP'),
  ('the', 'DET'),
  ('Big', 'NOUN'),
  ('Board', 'NOUN'),
  ('trading', 'NOUN'),
  ('floor', 'NOUN'),
  (',', '.'),
  ('says', 'VERB'),
  ('0', 'X'),
  ('*T*-3', 'X'),
  ('one', 'NUM'),
  ('specialist', 'NOUN'),
  ('there', 'ADV'),
  ('.', '.')],
 [('Gunmen', 'NOUN'),
  ('in', 'ADP'),
  ('Lebanon', 'NOUN'),
  ('assassinated', 'VERB'),
  ('a', 'DET'),
  ('Saudi', 'NOUN'),
  ('Arabian', 'NOUN'),
  ('Embassy', 'NOUN'),
  ('employee', 'NOUN'),
  (',', '.'),
  ('and', 'CONJ'),
  ('the', 'DET'),
  ('pro-Iranian', 'ADJ'),
  ('Islamic', 'NOUN'),
  ('Jihad', 'NOUN'),
  ('took', 'VERB'),
  ('responsibility', 'NOUN'),
  ('for', 'AD

In [49]:
# tagging the test sentences using Vanilla Viterbi
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Vanilla Viterbi accuracy (in %): ", accuracy)

Time taken in seconds:  81.50230836868286
Vanilla Viterbi accuracy (in %):  86.74698795180723


In [53]:
# checking the incorrectly tagged words
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('the', 'DET'), (('sell', 'VERB'), ('sell', 'NOUN'))],
 [('specialist', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('in', 'ADP'), (('Lebanon', 'DET'), ('Lebanon', 'NOUN'))],
 [('Lebanon', 'NOUN'), (('assassinated', 'NOUN'), ('assassinated', 'VERB'))],
 [('the', 'DET'), (('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [('by', 'ADP'), (('Riyadh', 'DET'), ('Riyadh', 'NOUN'))],
 [('Riyadh', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [('The', 'DET'), (('forthcoming', 'NOUN'), ('forthcoming', 'ADJ'))],
 [('a', 'DET'), (('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [('government', 'NOUN'),
  (('yen-denominated', 'NOUN'), ('yen-denominated', 'ADJ'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('investors', 'NOUN'), (('redeeming', 'NOUN'), ('redeeming', 'VERB'))],
 [('will', 'VERB'), (('convert', 'X'), ('convert', 'VERB'))]]

Upon checking the incorrectly tagged words, we can observe that the every time the Vanilla Viterbi algorithm encounters an unknown word, it assigns the tag of the word as 'X' since it is the first tag in the list of tags.

### Solve the problem of unknown words

#### Modified Viterbi I using probabilistic technique

In case of unknown words, the emission probability becomes zero. This issue can be resolved if we use the transition probability instead of the emission probability when there is an unknown word.

In [54]:
def ProbabilisticModifiedViterbi(words, train_bag = train_tagged_words):
    T = list(set([tag for word,tag in train_bag]))
    
    state = []
    for key, word in enumerate(words):
        p = [] # initialising list of probability column for each observation
        transition_p2 = [] # initialising list to store transition probabilities separately
        
        for tag in T:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]
            
            # storing transition probabilities
            transition_p2.append(transition_p)
            
            # calculating emission and state probabilities
            emission_p = w_given_t(words[key], tag)[0]/w_given_t(words[key], tag)[1]
            state_prob = emission_p * transition_p    
            p.append(state_prob)
            
          
        p_max = max(p)
        max_state = T[p.index(p_max)] 
        
        # using transition probability in case of unknown word (p_max is 0) 
        if p_max == 0:
            p_max = max(transition_p2)
            max_state = T[transition_p2.index(p_max)]
                           
        else:
            max_state = T[p.index(p_max)] 
        
        state.append(max_state)
    return list(zip(words, state))

In [55]:
# tagging the test sentences using Probabilistic Modified Viterbi
start = time.time()
tagged_seq = ProbabilisticModifiedViterbi(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Probabilistic Modified Viterbi accuracy (in %): ", accuracy)

Time taken in seconds:  83.18158626556396
Probabilistic Modified Viterbi accuracy (in %):  92.16867469879519


Above, we can see that using transition probability whenever the emission probability is zero (in case of unknown words) has caused the accuracy of the test set to improve significantly. We can further fine-tune this algorithm by using weighted transition probabilities. This is because the new algorithm will also consider the fact that some of the POS tags have a higher probability of occurence compared to others.

In [56]:
# calculating occurence probability of each tag
p_tag = []
len_train_tag = len([t for w,t in train_tagged_words])
for tag in T:
    temp = [tag for w,t in train_tagged_words if t == tag]
    p_tag.append((tag,len(temp)/len_train_tag))
p_tag

[('X', 0.06597146107996953),
 ('CONJ', 0.022380191860040293),
 ('NUM', 0.03511518909383188),
 ('PRT', 0.03204626353093456),
 ('DET', 0.08631614108706771),
 ('ADP', 0.09798640904393574),
 ('.', 0.11618075345254125),
 ('PRON', 0.027338489963360788),
 ('NOUN', 0.286756646729089),
 ('ADJ', 0.06328876084301506),
 ('ADV', 0.03185837012912452),
 ('VERB', 0.13476132318708964)]

In [57]:
# fine-tuning the ProbabilisticModifiedViterbi function
def ModifiedViterbi_1(words, train_bag = train_tagged_words):
    T = list(set([tag for word,tag in train_bag]))
    state = []
    
    for key, word in enumerate(words):
        p = [] # initialising list of probability column for each observation
        transition_p2 = [] # initialising list to store weighted transition probabilities
        
        for tag in T:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]
            
            # weighted transition probabilities of each tag
            tag_weight = [weight for tag_name, weight in p_tag if tag_name == tag ]
            
            # compute the transition prob weighted by tag occurance probability.
            transition_p = tag_weight[0]*transition_p             
            transition_p2.append(transition_p)
            
            # calculating emission and state probabilities
            emission_p = w_given_t(words[key], tag)[0]/w_given_t(words[key], tag)[1]
            state_prob = emission_p * transition_p    
            p.append(state_prob)
            
            
        p_max = max(p)
        max_state = T[p.index(p_max)] 
        
        # using weighted transmission probability in case of unknown word (p_max is 0)
        if p_max == 0:
            p_max = max(transition_p2)
            max_state = T[transition_p2.index(p_max)]
                           
        else:
            max_state = T[p.index(p_max)] 
        
        state.append(max_state)
    return list(zip(words, state))

In [58]:
# tagging the test sentences using Modified Viterbi I
start = time.time()
tagged_seq = ModifiedViterbi_1(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Modified Viterbi I accuracy (in %): ", accuracy)

Time taken in seconds:  79.11043095588684
Modified Viterbi I accuracy (in %):  92.16867469879519


In [59]:
# checking the incorrectly tagged words
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('printers', 'NOUN'), (('start', 'NOUN'), ('start', 'VERB'))],
 [('*T*-3', 'X'), (('one', 'NOUN'), ('one', 'NUM'))],
 [('specialist', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('Lebanon', 'NOUN'), (('assassinated', 'NOUN'), ('assassinated', 'VERB'))],
 [('the', 'DET'), (('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [('to', 'PRT'), (('avenge', 'NOUN'), ('avenge', 'VERB'))],
 [('The', 'DET'), (('forthcoming', 'NOUN'), ('forthcoming', 'ADJ'))],
 [('a', 'DET'), (('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [('10-year', 'NUM'), (('Japanese', 'NOUN'), ('Japanese', 'ADJ'))],
 [('government', 'NOUN'),
  (('yen-denominated', 'NOUN'), ('yen-denominated', 'ADJ'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('investors', 'NOUN'), (('redeeming', 'NOUN'), ('redeeming', 'VERB'))],
 [('will', 'VERB'), (('convert', 'NOUN'), ('convert', 'VERB'))]]

We can see that the accuracy of Modified Viterbi I has much improved compared to the Vanilla Viterbi. There are several instances where the words which were incorrectly tagged by Vanilla Viterbi has been correctly tagged by Modified Viterbi I.
    For example:
- The words 'printers', 'Gunmen', 'Arabian', 'Islamic', 'sweepstakes', 'Card' etc. have been correctly tagged as 'NOUN'
- ''s' has been correctly identified as 'PRT'

#### Modified Viterbi II using rule-based backoff method

Apart from the probabilistic method, we can also modify the Vanilla Viterbi algorithm so as to backoff to a rule-based tagger every time it comes across an unknown word. We can determine the tags of most of the unknown words using a rule-based tagger which uses regex.

In [60]:
# creating a rule-based tagger using regex
regex = [
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),    # cardinal numbers
    (r'.*ed$', 'VERB'),                  # past tense
    (r'.*\'s$', 'NOUN'),                 # possessive nouns
    (r'.*es$', 'VERB'),                  # verb   
    (r'.*s$', 'NOUN'),                   # plural nouns
    (r'.*ing$', 'VERB'),                 # gerund
    (r'.*', 'NOUN'),                     # nouns
    (r'\*T?\*?-[0-9]+$', 'X'),           # X    
]

Regex_tagger = nltk.RegexpTagger(regex)

In [68]:
def RuleBasedModifiedViterbi(words, train_bag = train_tagged_words):
    T = list(set([tag for word,tag in train_bag]))
    state = []
    
    for key, word in enumerate(words):
        p = [] #initialising list of probability column for each observation
                
        for tag in T:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]
            
            # computing emission and state probabilities
            emission_p = w_given_t(words[key], tag)[0]/w_given_t(words[key], tag)[1]
            state_prob = emission_p * transition_p    
            p.append(state_prob)
            
            
        p_max = max(p)
        max_state = Regex_tagger.tag([word])[0][1]
        
        # backing of to rule-based tagger in case of unknown word (p_max is 0)
        if p_max == 0:
            max_state = Regex_tagger.tag([word])[0][1]
                           
        else:
            if max_state != 'X':
                max_state = T[p.index(p_max)] 
        
        state.append(max_state)
    return list(zip(words, state))

In [62]:
# tagging the test sentences using Rule-based Modified Viterbi
start = time.time()
tagged_seq = RuleBasedModifiedViterbi(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Rule-based Modified Viterbi accuracy (in %): ", accuracy)

Time taken in seconds:  82.93318819999695
Rule-based Modified Viterbi accuracy (in %):  92.7710843373494


In [63]:
# checking incorrectly tagged words
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('the', 'DET'), (('sell', 'VERB'), ('sell', 'NOUN'))],
 [('specialist', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('the', 'DET'), (('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [('the', 'DET'), (('slaying', 'VERB'), ('slaying', 'NOUN'))],
 [('to', 'PRT'), (('avenge', 'NOUN'), ('avenge', 'VERB'))],
 [('the', 'DET'), (('beheading', 'VERB'), ('beheading', 'NOUN'))],
 [('a', 'DET'), (('sweepstakes', 'VERB'), ('sweepstakes', 'NOUN'))],
 [('The', 'DET'), (('forthcoming', 'VERB'), ('forthcoming', 'ADJ'))],
 [('a', 'DET'), (('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [('government', 'NOUN'),
  (('yen-denominated', 'VERB'), ('yen-denominated', 'ADJ'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('will', 'VERB'), (('convert', 'NOUN'), ('convert', 'VERB'))]]

We can see that the Rule-based Modified Viterbi is giving better results in terms of tagging accuracy compared to the Vanilla Viterbi as well as the Modified Viterbi I. However, we can once again fine-tune this algorithm by combining the Modified Viterbi I with the Rule-based Modified Viterbi. The final algorithm will also give an output of 'NN' instead of an 'X' when any of the rules are not satisfied.

In [64]:
# creating a new rule-based tagger using regex for words that do not satisfy any rules
regex2 = [
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),    # cardinal numbers
    (r'.*ed$', 'VERB'),                  # past tense
    (r'.*\'s$', 'NOUN'),                 # possessive nouns
    (r'.*es$', 'VERB'),                  # verb   
    (r'.*s$', 'NOUN'),                   # plural nouns
    (r'.*ing$', 'VERB'),                 # gerund
    (r'.*', 'NOUN'),                     # nouns
    (r'\*T?\*?-[0-9]+$', 'X'),           # X    
    (r'.*', 'NN')                        # default
]

Regex_tagger2 = nltk.RegexpTagger(regex2)

In [69]:
# fine-tuning the RuleBasedModifiedViterbi function
def ModifiedViterbi_2(words, train_bag = train_tagged_words):
    T = list(set([tag for word,tag in train_bag]))
    state = []
    
    for key, word in enumerate(words):
        p = [] # initialising list of probability column for each observation
        transition_p2 = [] # initialising list to store weighted transition probabilities
        
        for tag in T:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]
            
            # weighted transition probabilities of each tag
            tag_weight = [weight for tag_name, weight in p_tag if tag_name == tag ]
            
            # compute the transition prob weighted by tag occurance probability
            transition_p = tag_weight[0]*transition_p             
            transition_p2.append(transition_p)
            
            # calculating emission and state probabilities
            emission_p = w_given_t(words[key], tag)[0]/w_given_t(words[key], tag)[1]
            state_prob = emission_p * transition_p    
            p.append(state_prob)
            
            
        p_max = max(p)
        max_state = T[p.index(p_max)] 
        
        if p_max == 0:
            max_state = Regex_tagger2.tag([word])[0][1]
            
            # finding tag with highest transition probability when any of the rules are not satisfied in the rule-based tagger
            if max_state == 'NN':
                p_max = max(transition_p2)
                max_state = T[transition_p2.index(p_max)]
            
            
        else:
            if max_state != 'X':
                max_state = T[p.index(p_max)] 
        
        state.append(max_state)
    return list(zip(words, state))

In [70]:
# tagging the test sentences using Modified Viterbi II
start = time.time()
tagged_seq = ModifiedViterbi_2(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Modified Viterbi II accuracy (in %): ", accuracy)

Time taken in seconds:  77.87945890426636
Modified Viterbi II accuracy (in %):  91.56626506024097


In [67]:
# checking incorrectly tagged words
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('printers', 'NOUN'), (('start', 'NOUN'), ('start', 'VERB'))],
 [('*T*-3', 'X'), (('one', 'NOUN'), ('one', 'NUM'))],
 [('specialist', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('the', 'DET'), (('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [('the', 'DET'), (('slaying', 'VERB'), ('slaying', 'NOUN'))],
 [('to', 'PRT'), (('avenge', 'NOUN'), ('avenge', 'VERB'))],
 [('the', 'DET'), (('beheading', 'VERB'), ('beheading', 'NOUN'))],
 [('a', 'DET'), (('sweepstakes', 'VERB'), ('sweepstakes', 'NOUN'))],
 [('The', 'DET'), (('forthcoming', 'VERB'), ('forthcoming', 'ADJ'))],
 [('a', 'DET'), (('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [('10-year', 'NUM'), (('Japanese', 'NOUN'), ('Japanese', 'ADJ'))],
 [('government', 'NOUN'),
  (('yen-denominated', 'VERB'), ('yen-denominated', 'ADJ'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('will', 'VERB'), (('convert', 'NOUN'), ('convert', 'VERB'))]]

We see that the accuracy of Modified Viterbi II is slightly decreased than that of Modified Viterbi I. This can be due to the random samples taken from the test set. However, it is still greater than that of Vanilla Viterbi.
- 'redeeming' and 'assassinated' has been correctly tagged as a 'VERB'
- ''s' has been correctly assigned the tag of 'PRT'
- 'Riyadh' and 'Lebanon' is correctly tagged as 'NOUN' by Modified Viterbi II

We will have to run the modified Viterbi algorithms on the complete test set in order to make any concrete inferences in terms of accuracy.

##### Running the Vanilla Viterbi, Modified Viterbi I and Modified Viterbi II on the complete test set

In [71]:
test_tagged_words = [y[0] for x in test_data for y in x]
test_run_base = [y for x in test_data for y in x]

In [72]:
# Vanilla Viterbi
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Vanilla Viterbi accuracy (in %): ", accuracy)

Time taken in seconds:  2284.6522080898285
Vanilla Viterbi accuracy (in %):  91.0600779167521


In [73]:
# Modified Viterbi I
start = time.time()
tagged_seq = ModifiedViterbi_1(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Modified Viterbi I accuracy (in %): ", accuracy)

Time taken in seconds:  2161.5463399887085
Modified Viterbi I accuracy (in %):  93.50010252204224


In [74]:
# Modified Viterbi II
start = time.time()
tagged_seq = ModifiedViterbi_2(test_tagged_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = (len(check)/len(tagged_seq))*100
print("Modified Viterbi II accuracy (in %): ", accuracy)

Time taken in seconds:  2252.8400683403015
Modified Viterbi II accuracy (in %):  94.54582735288088


While the accuracy of Modified Viterbi I is better than Vanilla Viterbi, we can see above that the accuracy of Modified Viterbi II is better than that of both Modified Viterbi I and Vanilla Viterbi.

### Evaluating tagging accuracy on sample test sentences

In [75]:
path = input("Enter the path to data file: ") # Enter the path to the dataset file

Enter the path to data file: D:\upGrad\.CONTENT\Assignments\NLP Assignment


In [111]:
sample_data = open(path+'/Test_sentences.txt').read() # the path entered above will be used to read the file
sample_text = sample_data.splitlines()
open(path+'/Test_sentences.txt').close()
sample_text

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta',
 'I would like to see flights from Denver to Philadelphia.',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
 'NASA invited social media users to experienc

In [112]:
sample_words = [word for sentence in sample_text for word in sentence.split()]

In [113]:
# tagging the sample test sentences using Vanilla Viterbi
start = time.time()
Viterbi_sample_tagged_seq = Viterbi(sample_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

Time taken in seconds:  83.1683452129364


In [114]:
Viterbi_sample_tagged_seq

[('Android', 'X'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'X'),
 ('Android', 'X'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'X'),
 ('worldwide', 'X'),
 ('on', 'ADP'),
 ('smartphones', 'X'),
 ('since', 'ADP'),
 ('2011', 'X'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'X'),
 ('Google', 'X'),
 ('and', 'CONJ'),
 ('Twitter', 'X'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'X'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'X'),
 ('firehose.', 'X'),
 ('Twitter', 'X'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'X'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET'),
 ('users', 'NOUN'),
 ('post', 'NOUN'),
 ('

In [109]:
# tagging the sample test sentences using Modified Viterbi II
start = time.time()
ModifiedViterbi_sample_tagged_seq = ModifiedViterbi_2(sample_words)
end = time.time()
difference = end-start

# printing time taken and accuracy
print("Time taken in seconds: ", difference)

Time taken in seconds:  81.76875805854797


In [110]:
ModifiedViterbi_sample_tagged_seq

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The tagging accuracies in the given dataset are:
- Vanilla Viterbi : 91.06%

- Modified Viterbi I : 93.50%

- Modified Viterbi II : 94.55%

This shows that by using modified viterbi, we can significantly improve our tagging accuracy on a given text corpus.

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

There are several cases which were incorrectly identified by the Vanilla Viterbi POS tagger but were correctly identified by the Modified Viterbi POS tagger. Some of these cases are:

When using Vanilla Viterbi, we can see than some of the words have been incorrectly tagged such as:
- '2018', '2011', and '2015' have been tagged as 'X'
- 'NASA', 'Android', 'Google', 'worldwide', 'OS', 'smartphones', 'ICESAT-2' have been tagged as 'X'
- 'invited' and 'arriving' has been tagged as 'X'

On the other hand, the Modified Viterbi II tagger has correctly tagged the following:
- '2018', '2011', and '2015' have been tagged as 'NUM'
- 'NASA', 'Android', 'Google', 'worldwide', 'OS', 'smartphones', 'ICESAT-2' have been tagged as 'NOUN'
- 'invited' and 'arriving' has been tagged as 'VERB'